## ASHRAE 140-2023

Section 7: CASE 600

In [1]:
import OpenSimula as osm
import pandas as pd
import numpy as np

sim = osm.Simulation()
pro = sim.new_project("pro")
pro.read_json("case_600.json")

Reading project data from file: case_600.json
Reading completed.
Checking project: Case 600
ok


In [2]:
pro.component("Building").draw_pyvista()

Widget(value='<iframe src="http://localhost:52597/index.html?ui=P_0x1760eabd0_0&reconnect=auto" class="pyvista…

In [3]:
pro.simulate()

Simulating Case 600: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [4]:
# Heating and Cooling Loads
annual_heating = pro.component("space_1").variable("Q_heating").values.sum()/1e6
annual_cooling = pro.component("space_1").variable("Q_cooling").values.sum()/1e6
peak_heating = pro.component("space_1").variable("Q_heating").values.max()/1000
i_peak_heating = np.argmax(pro.component("space_1").variable("Q_heating").values)
peak_cooling = pro.component("space_1").variable("Q_cooling").values.max()/1000
i_peak_cooling = np.argmax(pro.component("space_1").variable("Q_cooling").values)

print("Anual Heating (MWh): ", annual_heating)
print("Anual Coolling (MWh): ",annual_cooling)
print("Peak Heating (kW): ", peak_heating)
print("Peak Cooling (kW): ", peak_cooling)
print("Peak Heating Date: ",pro.dates()[i_peak_heating])
print("Peak Cooling Date: ",pro.dates()[i_peak_cooling])

Anual Heating (MWh):  4.544157886678226
Anual Coolling (MWh):  5.468623665619255
Peak Heating (kW):  3.2653091944198582
Peak Cooling (kW):  5.577009402662208
Peak Heating Date:  2001-12-31 23:30:00
Peak Cooling Date:  2001-01-22 13:30:00


In [5]:
# Solar Radiation
hor_sol_rad = pro.component("roof_wall").variable("E_dir").values.sum()/1000 + pro.component("roof_wall").variable("E_dif").values.sum()/1000
north_sol_rad = pro.component("north_wall").variable("E_dir").values.sum()/1000 + pro.component("north_wall").variable("E_dif").values.sum()/1000
east_sol_rad = pro.component("east_wall").variable("E_dir").values.sum()/1000 + pro.component("east_wall").variable("E_dif").values.sum()/1000
south_sol_rad = pro.component("south_wall").variable("E_dir").values.sum()/1000 + pro.component("south_wall").variable("E_dif").values.sum()/1000
west_sol_rad = pro.component("west_wall").variable("E_dir").values.sum()/1000 + pro.component("west_wall").variable("E_dif").values.sum()/1000
window_trans =  pro.component("south_window_1").variable("E_dir_tra").values.sum()/1000 + pro.component("south_window_1").variable("E_dif_tra").values.sum()/1000

print("Horizontal solar rad. (kWh/m²): ",hor_sol_rad)
print("North solar rad. (kWh/m²): ", north_sol_rad)
print("East solar rad. (kWh/m²): ",east_sol_rad)
print("South solar rad. (kWh/m²): ",south_sol_rad)
print("West solar rad. (kWh/m²): ", west_sol_rad)
print ("Window transmitted solar rad. (kWh/m²): ",window_trans)

Horizontal solar rad. (kWh/m²):  1670.2199999999998
North solar rad. (kWh/m²):  478.7461667889758
East solar rad. (kWh/m²):  1016.9676383985318
South solar rad. (kWh/m²):  1283.0501974539516
West solar rad. (kWh/m²):  919.7928583336316
Window transmitted solar rad. (kWh/m²):  760.9337566121537


In [6]:
# Sky temperature
t_sky_average = pro.component("Denver").variable("sky_temperature").values.mean()
t_sky_min = pro.component("Denver").variable("sky_temperature").values.min()
i_t_sky_min = np.argmin(pro.component("Denver").variable("sky_temperature").values)
t_sky_max = pro.component("Denver").variable("sky_temperature").values.max()
i_t_sky_max= np.argmax(pro.component("Denver").variable("sky_temperature").values)

print("Average sky temp (ºC): ",t_sky_average)
print("Min. sky temp (ºC): ",t_sky_min)
print("Max. sky temp (ºC): ",t_sky_max)
print("Min. sky temp date: ",pro.dates()[i_t_sky_min])
print("Max. sky temp date: ",pro.dates()[i_t_sky_max])


Average sky temp (ºC):  -1.985010322048502
Min. sky temp (ºC):  -37.95756585462965
Max. sky temp (ºC):  26.05027737992407
Min. sky temp date:  2001-12-31 23:30:00
Max. sky temp date:  2001-08-04 14:30:00


In [7]:
df_month_loads = pd.DataFrame()
aux = pro.component("space_1").variable_dataframe(frequency="M",value="sum")
df_month_loads["Heating (kWh)"] = aux["Q_heating [W]"]/1000
df_month_loads["Cooling (kWh)"] = aux["Q_cooling [W]"]/1000
df_month_loads

,Heating (kWh),Cooling (kWh)
date,,
2001-01-31,729.440707,420.018853
2001-02-28,711.267805,306.716651
2001-03-31,503.327761,408.436628
2001-04-30,523.256775,228.165980
2001-05-31,150.650834,352.033464
2001-06-30,14.584467,558.740121
2001-07-31,14.741226,548.912851
2001-08-31,10.795788,634.800446
2001-09-30,88.840245,671.210313


In [8]:
def month_max(df,variable):
    dates = []
    values = []
    for mes in range(1,13):
        df_mes = df[df["date"].dt.month == mes]
        values.append(df_mes[variable].max())
        i = df_mes[variable].idxmax()
        dates.append(df_mes["date"][i])
    return (dates,values)


df_month_peak_loads = pd.DataFrame()
aux = pro.component("space_1").variable_dataframe(frequency="M",value="max")
dates_heating, values_heating = month_max(pro.component("space_1").variable_dataframe(),"Q_heating [W]")
dates_cooling, values_cooling = month_max(pro.component("space_1").variable_dataframe(),"Q_cooling [W]")
df_month_peak_loads["Heating (kW)"] = values_heating
df_month_peak_loads["Heating (kW)"] = df_month_peak_loads["Heating (kW)"]/1000
df_month_peak_loads["Cooling (kW)"] = values_cooling
df_month_peak_loads["Cooling (kW)"] = df_month_peak_loads["Cooling (kW)"]/1000
df_month_peak_loads["Heating dates"] = dates_heating
df_month_peak_loads["Cooling dates"] = dates_cooling

df_month_peak_loads


,Heating (kW),Cooling (kW),Heating dates,Cooling dates
0,2.966563,5.577009,2001-01-04 03:30:00,2001-01-22 13:30:00
1,3.108103,4.535474,2001-02-09 05:30:00,2001-02-13 13:30:00
2,2.209792,4.385937,2001-03-17 05:30:00,2001-03-09 13:30:00
3,2.389095,3.714729,2001-04-11 04:30:00,2001-04-07 13:30:00
4,1.674855,3.081579,2001-05-01 03:30:00,2001-05-10 13:30:00
5,0.881789,3.322215,2001-06-09 04:30:00,2001-06-14 13:30:00
6,0.602972,3.408431,2001-07-26 04:30:00,2001-07-29 13:30:00
7,0.581639,4.226138,2001-08-17 05:30:00,2001-08-27 12:30:00
8,1.233698,4.995166,2001-09-28 23:30:00,2001-09-30 13:30:00
9,1.720580,5.297203,2001-10-15 04:30:00,2001-10-18 13:30:00


In [9]:
may_4_df = pd.DataFrame()
intervalo = ["2001-05-04","2001-05-05"]
aux = pro.component("roof_wall").variable_dataframe(interval=intervalo)
may_4_df["Hor. rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux = pro.component("south_wall").variable_dataframe(interval=intervalo)
may_4_df["South rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux = pro.component("west_wall").variable_dataframe(interval=intervalo)
may_4_df["West rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux =  pro.component("south_window_1").variable_dataframe(interval=intervalo)
may_4_df["Window trans. rad. (W/m²)"] = aux["E_dir_tra [W/m²]"] +  aux["E_dif_tra [W/m²]"]
aux =  pro.component("Denver").variable_dataframe(interval=intervalo)
may_4_df["Sky temp. (ºC)"] = aux["sky_temperature [°C]"]
aux =  pro.component("space_1").variable_dataframe(interval=intervalo)
may_4_df["Heating Load (kWh)"] = aux["Q_heating [W]"]/1000
may_4_df["Cooling Load (kWh)"] = aux["Q_cooling [W]"]/1000
#may_4_df.to_clipboard()
may_4_df

,Hor. rad. (W/m²),South rad. (W/m²),West rad. (W/m²),Window trans. rad. (W/m²),Sky temp. (ºC),Heating Load (kWh),Cooling Load (kWh)
2952,0.0,0.0,0.0,0.000000,-1.509956,1.056489,0.0
2953,0.0,0.0,0.0,0.000000,-0.897172,1.102460,0.0
2954,0.0,0.0,0.0,0.000000,-1.509956,1.165239,0.0
2955,0.0,0.0,0.0,0.000000,-1.509956,1.195050,0.0
2956,0.0,0.0,0.0,0.000000,-1.509956,1.212402,0.0
2957,15.0,9.0,9.0,5.452925,-1.509956,1.189191,0.0
2958,60.0,36.0,36.0,21.811700,-0.897172,1.048224,0.0
2959,111.0,66.6,66.6,40.351645,-0.782669,0.864120,0.0
2960,158.0,94.8,94.8,57.437476,-0.329572,0.630998,0.0
2961,195.0,117.0,117.0,70.888024,1.351987,0.348915,0.0


In [10]:
feb_1_df = pd.DataFrame()
intervalo = ["2001-02-01","2001-02-02"]
aux = pro.component("roof_wall").variable_dataframe(interval=intervalo)
feb_1_df["Hor. rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux = pro.component("south_wall").variable_dataframe(interval=intervalo)
feb_1_df["South rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux = pro.component("west_wall").variable_dataframe(interval=intervalo)
feb_1_df["West rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux =  pro.component("south_window_1").variable_dataframe(interval=intervalo)
feb_1_df["Window trans. rad. (W/m²)"] = aux["E_dir_tra [W/m²]"] +  aux["E_dif_tra [W/m²]"]
aux =  pro.component("Denver").variable_dataframe(interval=intervalo)
feb_1_df["Sky temp. (ºC)"] = aux["sky_temperature [°C]"]
aux =  pro.component("space_1").variable_dataframe(interval=intervalo)
feb_1_df["Heating Load (kWh)"] = aux["Q_heating [W]"]/1000
feb_1_df["Cooling Load (kWh)"] = aux["Q_cooling [W]"]/1000
feb_1_df.to_clipboard()
feb_1_df

,Hor. rad. (W/m²),South rad. (W/m²),West rad. (W/m²),Window trans. rad. (W/m²),Sky temp. (ºC),Heating Load (kWh),Cooling Load (kWh)
744,0.0,0.000000,0.000000,0.000000,-23.447622,2.201030,0.000000
745,0.0,0.000000,0.000000,0.000000,-23.872637,2.266501,0.000000
746,0.0,0.000000,0.000000,0.000000,-26.084956,2.329021,0.000000
747,0.0,0.000000,0.000000,0.000000,-27.912259,2.391551,0.000000
748,0.0,0.000000,0.000000,0.000000,-26.214853,2.386385,0.000000
749,0.0,0.000000,0.000000,0.000000,-26.910724,2.419689,0.000000
750,0.0,0.000000,0.000000,0.000000,-27.561544,2.410090,0.000000
751,38.0,191.323666,9.800000,97.314941,-25.722382,1.802636,0.000000
752,209.0,484.097954,34.900000,293.094977,-25.023357,0.375882,0.000000
753,383.0,694.887587,57.800000,457.894124,-24.047287,0.000000,0.000000


In [11]:
jul_14_df = pd.DataFrame()
intervalo = ["2001-07-14","2001-07-15"]
aux = pro.component("roof_wall").variable_dataframe(interval=intervalo)
jul_14_df["Hor. rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux = pro.component("south_wall").variable_dataframe(interval=intervalo)
jul_14_df["South rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux = pro.component("west_wall").variable_dataframe(interval=intervalo)
jul_14_df["West rad. (W/m²)"] = aux["E_dir [W/m²]"] +  aux["E_dif [W/m²]"]
aux =  pro.component("south_window_1").variable_dataframe(interval=intervalo)
jul_14_df["Window trans. rad. (W/m²)"] = aux["E_dir_tra [W/m²]"] +  aux["E_dif_tra [W/m²]"]
aux =  pro.component("Denver").variable_dataframe(interval=intervalo)
jul_14_df["Sky temp. (ºC)"] = aux["sky_temperature [°C]"]
aux =  pro.component("space_1").variable_dataframe(interval=intervalo)
jul_14_df["Heating Load (kWh)"] = aux["Q_heating [W]"]/1000
jul_14_df["Cooling Load (kWh)"] = aux["Q_cooling [W]"]/1000
jul_14_df.to_clipboard()
jul_14_df

,Hor. rad. (W/m²),South rad. (W/m²),West rad. (W/m²),Window trans. rad. (W/m²),Sky temp. (ºC),Heating Load (kWh),Cooling Load (kWh)
4656,0.0,0.000000,0.000000,0.000000,4.128365,0.000000,0.000000
4657,0.0,0.000000,0.000000,0.000000,3.648777,0.000000,0.000000
4658,0.0,0.000000,0.000000,0.000000,2.514234,0.000000,0.000000
4659,0.0,0.000000,0.000000,0.000000,1.463941,0.000000,0.000000
4660,2.0,1.200000,1.200000,0.727057,0.321597,0.028965,0.000000
4661,90.0,20.500000,20.500000,12.420551,2.948471,0.000000,0.000000
4662,275.0,46.000000,46.000000,27.870505,5.535130,0.000000,0.000000
4663,479.0,73.400000,73.400000,44.471632,7.321400,0.000000,0.000000
4664,668.0,163.387347,98.800000,66.464846,9.174255,0.000000,0.000000
4665,827.0,292.711019,119.200000,115.520609,12.000253,0.000000,0.546974
